In [212]:
#results[147719922.0][0]['builds'].head(1).gh_build_started_at.values[0]
#results[147719922.0][0]['builds'].head(1).gh_build_started_at.values[0]

dt_end = data_frame[data_frame.tr_build_id == 147719922].head(1).gh_build_started_at.values[0]
dt_start = results[147719922.0][1561]['builds'].head(1).gh_build_started_at.values[0]
import datetime
print(dt_end, dt_start)
delta = datetime.datetime(2016,7,27,10,34,49) - datetime.datetime(2015,12,17,17,58,40)
delta.days

2016-07-27 10:34:49 2015-12-17 17:58:40


222

In [91]:
result_dict.keys()


dict_keys([133121.0, 109571.0, 108038.0, 120839.0, 153415.0, 145923.0, 126229.0, 108310.0, 110041.0, 142360.0, 125080.0, 149784.0, 120608.0, 122403.0, 133818.0, 142122.0, 112903.0, 114476.0, 122159.0, 149137.0, 107065.0, 129082.0, 124479.0, 113888.0, 106306.0, 124483.0, 109153.0, 111178.0, 109132.0, 114511.0, 136285.0, 115806.0, 109663.0, 109409.0, 108391.0, 113257.0, 147307.0, 140908.0, 106863.0, 106866.0, 134262.0, 134207.0, 146302.0, 133301.0, 151939.0, 143749.0, 114311.0, 153031.0, 108682.0, 143243.0, 130445.0, 134287.0, 124305.0, 148630.0, 122519.0, 118680.0, 152516.0, 150170.0, 108699.0, 134050.0, 106907.0, 143780.0, 107176.0, 107947.0, 130476.0, 121263.0, 112304.0, 148147.0, 136116.0, 138677.0, 126648.0, 143034.0, 114876.0, 112068.0, 106951.0, 106060.0, 106446.0, 150523.0, 124880.0, 151715.0, 118740.0, 148693.0, 121814.0, 134871.0, 133080.0, 124121.0, 121564.0, 111839.0, 140256.0, 134114.0, 106875.0, 130021.0, 109033.0, 116719.0, 145651.0, 127732.0, 151030.0, 115963.0, 113916.0,

In [211]:
# Determine all failed prev for each starting point (passed with prev failed)

# Helper to find relevant data
def get_build_rows_by_build_id(tr_build_id):
    return org.where(lambda x: x.tr_build_id==tr_build_id)
    
# Result arr with custom data structure, contains the build id and the
result_dict = dict()
def search_for_prev_builds(row): # Determine all failed preds for a specific starting print
    if len(result_dict) > 100:
        return
    if (row.tr_build_id in result_dict.keys()):
        #print("Wrong state for build id {0}".format(row.tr_build_id))
        return
    if not row.tr_build_id > 0: #nan
        return

    result_dict[row.tr_build_id] = [] # List of previous builds
    # 126229 => 117642 (errored)
    # 117642 => 117385 (errored)
    # 117385 => NaN
    current_prev_build_id = row.tr_prev_build
    while(current_prev_build_id > 0):
        prev_build_rows = get_build_rows_by_build_id(current_prev_build_id)
        result_dict[row.tr_build_id].append({"build_id": current_prev_build_id, "builds": prev_build_rows})
        #print(row.tr_build_id, "=>", current_prev_build_id, "=>", prev_build_rows.tr_prev_build.count().compute())
        if(prev_build_rows.tr_prev_build.count().compute() > 0 ): # any content?
            print(row.tr_build_id, "=>", current_prev_build_id, "=>", prev_build_rows.tr_prev_build.count())
            current_prev_build_id = prev_build_rows.dropna().head(1).tr_prev_build.values[0]
            print("=>", current_prev_build_id)
        else:
            current_prev_build_id = 0
    return row

result_data = passed_whith_prev_failed[passed_whith_prev_failed.tr_prev_build.notnull()]
#result_data = result_data[result_data.tr_build_id == 126229]
result_data.apply(search_for_prev_builds, axis=1)


for key, value in result_dict.items():
    print(key, ",", len(value))
    #for thing in value:
    #    thing['builds'].dropna()

#get_build_rows_by_build_id(108038)

KeyboardInterrupt: 

In [92]:
result_dict[120317.0]

[{'build_id': 118678.0, 'builds': dd.DataFrame<where-7..., npartitions=127>},
 {'build_id': 118466.0, 'builds': dd.DataFrame<where-2..., npartitions=127>},
 {'build_id': 117980.0, 'builds': dd.DataFrame<where-b..., npartitions=127>},
 {'build_id': 114914.0, 'builds': dd.DataFrame<where-6..., npartitions=127>},
 {'build_id': 114012.0, 'builds': dd.DataFrame<where-2..., npartitions=127>},
 {'build_id': 113931.0, 'builds': dd.DataFrame<where-1..., npartitions=127>},
 {'build_id': 113311.0, 'builds': dd.DataFrame<where-f..., npartitions=127>},
 {'build_id': 111887.0, 'builds': dd.DataFrame<where-1..., npartitions=127>}]

In [103]:
for item in result_dict[120317.0]:
    print(item["build_id"])
    print(item["builds"].dropna().head(1).tr_prev_build.values[0])

118678.0
118466.0
118466.0
117980.0
117980.0
114914.0
114914.0
114012.0
114012.0
113931.0
113931.0
113311.0
113311.0
111887.0
111887.0


C:\Users\schlauch\AppData\Local\Continuum\Anaconda3\lib\site-packages\dask\dataframe\core.py:3637: UserWarning: Insufficient elements for `head`. 1 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


IndexError: index 0 is out of bounds for axis 0 with size 0